# Laboratorio 6 Parte 2

### Reducción de dimensión: PCA y LDA

### 2019-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

## Estudiantes

#### Primer Integrante: Jairo David Campaña Rosero
#### Segundo Integrante: Alejandro Mesa Gómez

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de extracción de características (PCA y LDA) y usar SVM para resolver un problema de clasificación multietiqueta o multiclase.


Antes de iniciar a ejecutar las celdas, debe instalar la librería mlxtend que usaremos para los laboratorios de reducción de dimensión.
Para hacerlo solo tiene que usar el siguiente comando: 
`!pip install mlxtend`
También puede consultar la guía oficial de instalación
    de esta librería: https://rasbt.github.io/mlxtend/installation/





In [3]:
!pip install mlxtend

In [50]:
import warnings
warnings.filterwarnings("ignore")


import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import PrincipalComponentAnalysis as PCA
from mlxtend.feature_extraction import LinearDiscriminantAnalysis as LDA

import time


Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography



Analice la base de datos, sus características, su variable de salida y el contexto del problema.



Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [36]:
#cargamos la bd de entrenamiento
db = np.loadtxt('DB/DB_Fetal_Cardiotocograms.txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

#Para darle formato de entero a la variable de salida

Y_l = []
for i in Y:
    Y_l.append(int(i))
Y = np.asarray(Y_l)

print ("Dimensiones de la base de datos de entrenamiento. dim de X: " + str(np.shape(X)) + "\tdim de Y: " + str(np.shape(Y)))


Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126,)


## Ejercicio 1: Entrenamiento sin extracción de características

En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [45]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

#Para calcular el costo computacional
tiempo_i = time.time()

#Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

#Implemetamos la metodología de validación

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  

    #Aquí se entran y se valida el modelo sin hacer selección de características
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1

print("\nError de validación sin aplicar extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
print("\nEficiencia en validación sin aplicar extracción: " + str((1-np.mean(Errores))*100) + "%" )
print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

#print str(ypred)
#print str(y_test) 


Error de validación sin aplicar extracción: 0.07712817787226504 +/- 0.05442325724156325

Eficiencia en validación sin aplicar extracción: 92.2871822127735%


Tiempo total de ejecución: 0.586920976638794 segundos.




1.1 Cuando se aplica PCA ¿es necesario estandarizar los datos? Si, No y por qué? En qué consiste dicha estandarización?

R/: Es necesario estandarizar datos en el caso de que estos se encuentren en diferente escala(diferentes desviaciones estandar). Esto es debido a que PCA es un problema de maximizacion de varianza, el cual calcula un nuevo eje basado en la desviacion estandar de las variables, una variable con mayor desviacion estandar aportara mucho más  en dicho calculo del eje que una con menor, por lo tanto, es necesario estandarizarlas para llevars a tener la misma desviacion estandar y aportar de igual manera al calculo del nuevo eje.
    
1.2 La proyección de los datos que realiza PCA busca optimizar un medida, ¿Cuál? Explique.

R/: PCA tiene el objetivo principalmente de maximizar la varianza total de tal forma que se minimice la distancia entre los datos reales y proyecciones de estos sobre el nuevo eje generado.

## Ejercicio 2: Entrenamiento con extracción de características

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de extracción de características. https://rasbt.github.io/mlxtend/user_guide/feature_extraction/

In [4]:
'''
Feature Extraction Function
#Recibe 2 parámetros: 
1. el tipo de método de extracción (pca o lda como string),
2. el número componentes (para pca) o el número de discriminantes (para lda)

#Para este laboratorio solo se le pedirá trabajar con PCA, LDA es opcional.
'''

def extract_features(tipo, n):
    
    if tipo == 'pca':
    
        ext = PCA(n_components=n)
    
        return ext

    elif tipo == 'lda':
        
        ext = LDA(n_discriminants=n)
        
        return ext
    
    else:
        print ("Ingrese un método válido (pca o lda)\n")

In [5]:

#Para calcular el costo computacional
tiempo_i = time.time()

#Estandarizamos los datos
X = standardize(X)

#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    #Aquí se aplica la extracción de características por PCA
    #Complete el código
    
    ex = extract_features('pca',10)

    #Fit de PCA
    ex = ex.fit(X)
    
    #Transforme las variables y genere el nuevo espacio de características de menor dimensión
    X_ex = ex.transform(X)
    
    
    
    #Se aplica CV-10
    
    X_train, X_test = X_ex[train_index], X_ex[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
   
    #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1
        

print("\nError de validación aplicando extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print("\nEficiencia en validación aplicando extracción: " + str((1-np.mean(Errores))*100) + "%" )

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")



Error de validación aplicando extracción: 0.08932589246168837 +/- 0.06387058313574974

Eficiencia en validación aplicando extracción: 91.06741075383115%


Tiempo total de ejecución: 3.631760358810425 segundos.


## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de componentes principales a tener en cuenta (use 2, 10, 19 y 21 componentes principales) para PCA y complete la siguiente tabla de resultados:

In [6]:
def train(car, X, tipo):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Estandarizamos los datos
    X = standardize(X)

    #Implemetamos la metodología de validación cross validation con 10 folds

    Errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)

    for train_index, test_index in kf.split(X):

        #Aquí se aplica la extracción de características por PCA
        #Complete el código

        ex = extract_features(tipo, car)

        #Fit de PCA
        ex = ex.fit(X)

        #Transforme las variables y genere el nuevo espacio de características de menor dimensión
        X_ex = ex.transform(X)


        #Se aplica CV-10

        X_train, X_test = X_ex[train_index], X_ex[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA

        ######

        # Entrenamiento el modelo.
        model = clf.fit(X_train,y_train)

        # Validación del modelo
        ypred = model.predict(X_test)

        #######

        Errores[j] = classification_error(ypred, y_test)
        j+=1

    return str(np.mean(Errores)), str(np.std(Errores)),str((1-np.mean(Errores))*100) + "%",str(time.time()-tiempo_i) + " segundos."

In [7]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin extracción','SVM + PCA','SVM + PCA','SVM + PCA','SVM + PCA']),
    '# de características seleccionadas' : pd.Series(['N/A',2,10,19,21]),
   })
err_val_ = []
ic_ = []
efic_ = []
texec_ = []

for i in zip(df_types['# de características seleccionadas']):
    if(i[0] == 'N/A'):
        err_val_.append("0.07712817787226504")
        ic_.append("0.05442325724156325")
        efic_.append("92.66365488528656%")
        texec_.append("0.4625818729400635 segundos")
    else:
        err_val, ic, efic, texec = train(i[0], X, "pca")
        err_val_.append(err_val)
        ic_.append(ic)
        efic_.append(efic)
        texec_.append(texec)
    
df_types["Error de validación"] = err_val_
df_types["IC(std)"] = ic_
df_types["Eficiencia"] = efic_
df_types["Tiempo de ejecución"] = texec_

df_types.set_index(['Tecnica','# de características seleccionadas'], inplace=True)

#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [8]:
qgrid_widget.get_changed_df()

Error de validación  \
Tecnica            # de características seleccionadas                        
SVM sin extracción N/A                                 0.07712817787226504   
SVM + PCA          2                                   0.21483745238727964   
                   10                                  0.08932589246168837   
                   19                                  0.07383514926034193   
                   21                                  0.07336345114713438   

                                                                    IC(std)  \
Tecnica            # de características seleccionadas                         
SVM sin extracción N/A                                  0.05442325724156325   
SVM + PCA          2                                     0.1709424253642628   
                   10                                   0.06387058313574974   
                   19                                   0.04357915071494415   
                   21                                  0.043533331744924374   

                                                               Eficiencia  \
Tecnica            # de características seleccionadas                       
SVM sin extracción N/A                                 92.66365488528656%   
SVM + PCA          2                                   78.51625476127204%   
                   10                                  91.06741075383115%   
                   19                                   92.6164850739658%   
                   21                                  92.66365488528656%   

                                                                Tiempo de ejecución  
Tecnica            # de características seleccionadas                                
SVM sin extracción N/A                                  0.4625818729400635 segundos  
SVM + PCA          2                                   3.6112253665924072 segundos.  
                   10                                   3.141091823577881 segundos.  
                   19                                   3.299067735671997 segundos.  
                   21                                  3.3839051723480225 segundos.

3.2 Analizando los resultados del punto anterior que puede decir de la viabilidad de aplicar PCA para hacer reducción de dimensión en este problema?

R/: No es apropiado, ya que tanto el error como la eficiencia fueron mejores en la maquina de soporte sin extracción de características salvo en 21 características, donde el error es ligeramente menor pero a un nivel despreciable. Además, conforme se elige un numero menor de características el error aumenta, lo cual es una clara señal de que PCA no es util en este caso.


3.3 Explique en sus palabras la principal ventaja que tiene LDA sobre PCA para resolver problemas de clasificación.

R/: 

En el caso de clasificación, PCA, en su proceso de selección puede terminar traslapando clases al proyectar los datos a una dimensión menor, haciendo así que el problema aumente en complejidad. LDA busca los subespacios de características que maximicen la separación de clases, maxizando a su vez también la varianza intra clase, mientras que PCA busca la dirección de los vectores que maximice la varianza de los datos.



3.3 Explique en sus palabras las diferencias que existen entre los métodos de selección de características y los métodos de extracción de características vistos en el curso.

R/: Los metodos de extracción generan nuevas características para describir el comportamiento de los datos originales, mientras que los metodos de selección son tecnicas heurísticas que evaluan el desempeño del modelo debido a cada característica, por este motivo, es un proceso generalmente mucho más lento que extracción.


## * Ejercicio 4: LDA (Punto opcional)

En este punto hará uso del método <font color='blue'>extract_features</font>  que recibe como parámetro el tipo de extracción si es PCA o LDA y el número de discriminantes, una vez se haga la transformación de las carácteristicas, se continua con el procedimiento del modelo a entrenar, en este caso es SVM. Esta es documentación para LDA [link ](https://rasbt.github.io/mlxtend/user_guide/feature_extraction/LinearDiscriminantAnalysis/)

In [24]:
print(np.isnan(X).any())
print(np.isnan(Y).any())

print(np.isinf(X).any())
print(np.isinf(Y).any())

False
False
False
False


In [32]:
from mlxtend.data import iris_data
X, y = iris_data()
X = standardize(X)

lda = LDA(n_discriminants=2)
lda.fit(X, y)
X_lda = lda.transform(X)

In [33]:
X.shape

(150, 4)

In [34]:
y.shape

(150,)

In [19]:
Y = np.reshape(Y, (Y.shape[0],1))
Y.shape

(2126, 1)

In [20]:
X.shape

(2126, 22)

In [28]:
np.unique(Y)

array([1, 2, 3])

In [55]:
def train_lda(car, tipo, X, Y):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Estandarizamos los datos
    X = standardize(X)
        #Aquí se aplica la extracción de características por PCA
    #Complete el código

    ex = LDA(n_discriminants = car)

    #Fit de PCA
    ex = ex.fit(X,Y)

    #Transforme las variables y genere el nuevo espacio de características de menor dimensión
    X_ex = ex.transform(X)


    #Implemetamos la metodología de validación cross validation con 10 folds

    Errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)

    for train_index, test_index in kf.split(X):



        #Se aplica CV-10

        X_train, X_test = X_ex[train_index], X_ex[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA

        ######

        # Entrenamiento el modelo.
        model = clf.fit(X_train,y_train)

        # Validación del modelo
        ypred = model.predict(X_test)

        #######

        Errores[j] = classification_error(ypred, y_test)
        j+=1

    return str(np.mean(Errores)), str(np.std(Errores)),str((1-np.mean(Errores))*100) + "%",str(time.time()-tiempo_i) + " segundos."

In [56]:
train_lda(10,"",X,Y)

LinAlgError: Array must not contain infs or NaNs

In [51]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA2
from sklearn.preprocessing import scale

def train_lda2(car, tipo, X, Y):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Estandarizamos los datos
    X = scale(X)

    #Implemetamos la metodología de validación cross validation con 10 folds

    Errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)

    for train_index, test_index in kf.split(X):

        #Aquí se aplica la extracción de características por PCA
        #Complete el código

        ex = LDA2(n_components=10)
        #Fit de PCA
        ex = ex.fit(X,Y)

        #Transforme las variables y genere el nuevo espacio de características de menor dimensión
        X_ex = ex.transform(X)


        #Se aplica CV-10

        X_train, X_test = X_ex[train_index], X_ex[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA

        ######

        # Entrenamiento el modelo.
        model = clf.fit(X_train,y_train)

        # Validación del modelo
        ypred = model.predict(X_test)

        #######

        Errores[j] = classification_error(ypred, y_test)
        j+=1

    return str(np.mean(Errores)), str(np.std(Errores)),str((1-np.mean(Errores))*100) + "%",str(time.time()-tiempo_i) + " segundos."

In [52]:
train_lda2(2, "lda", X,Y)

('0.07474754185490302',
 '0.05435890079129884',
 '92.52524581450969%',
 '0.3339509963989258 segundos.')

In [14]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin extracción','SVM + LDA','SVM + LDA','SVM + LDA','SVM + LDA']),
    '# de discriminantes' : pd.Series(['N/A',2,10,19,21]),   })

err_val_ = []
ic_ = []
efic_ = []
texec_ = []

for i in zip(df_types['# de discriminantes']):
    if(i[0] == 'N/A'):
        err_val_.append("0.07712817787226504")
        ic_.append("0.05442325724156325")
        efic_.append("92.66365488528656%")
        texec_.append("0.4625818729400635 segundos")
    else:
        err_val, ic, efic, texec = train_lda(i[0], "lda")
        err_val_.append(err_val)
        ic_.append(ic)
        efic_.append(efic)
        texec_.append(texec)
    
df_types["Error de validación"] = err_val_
df_types["IC(std)"] = ic_
df_types["Eficiencia"] = efic_
df_types["Tiempo de ejecución"] = texec_

    
df_types["Error de validación"] = err_val_
df_types["IC(std)"] = ic_
df_types["Eficiencia"] = efic_
df_types["Tiempo de ejecución"] = texec_


df_types.set_index(['Tecnica','# de discriminantes'], inplace=True)

#df_types.sort_index(inplace=True)
qgrid_widget2 = qgrid.show_grid(df_types, show_toolbar=False)


TypeError: fit() missing 1 required positional argument: 'y'

In [15]:
qgrid_widget2.get_changed_df()

Error de validación  \
Tecnica            # de discriminantes                        
SVM sin extracción N/A                  0.07712817787226504   
SVM + LDA          2                    0.21483745238727964   
                   10                   0.08932589246168837   
                   19                   0.07383514926034193   
                   21                   0.07336345114713438   

                                                     IC(std)  \
Tecnica            # de discriminantes                         
SVM sin extracción N/A                   0.05442325724156325   
SVM + LDA          2                      0.1709424253642628   
                   10                    0.06387058313574974   
                   19                    0.04357915071494415   
                   21                   0.043533331744924374   

                                                Eficiencia  \
Tecnica            # de discriminantes                       
SVM sin extracción N/A                  92.66365488528656%   
SVM + LDA          2                    78.51625476127204%   
                   10                   91.06741075383115%   
                   19                    92.6164850739658%   
                   21                   92.66365488528656%   

                                                 Tiempo de ejecución  
Tecnica            # de discriminantes                                
SVM sin extracción N/A                   0.4625818729400635 segundos  
SVM + LDA          2                    3.3058104515075684 segundos.  
                   10                   3.1073663234710693 segundos.  
                   19                   3.0319924354553223 segundos.  
                   21                   2.9479408264160156 segundos.